<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [ ]:
#export

import numpy as np

from sklearn.base import BaseEstimator, RegressorMixin
from sklearn.utils.validation import check_X_y, check_array, check_is_fitted
from sklearn.utils import check_random_state

import mulearn.kernel as kernel
import mulearn.fuzzifier as fuzz
from mulearn.optimization import solve_optimization
from mulearn.optimization import solve_optimization_tensorflow

In [ ]:
# default_exp __init__

In [ ]:
#hide
from nbdev.showdoc import show_doc

In [ ]:
#export

class FuzzyInductor(BaseEstimator, RegressorMixin):

    def __init__(self,
                 c=1,
                 k=kernel.GaussianKernel(),
                 sample_generator=None,
                 fuzzifier=fuzz.ExponentialFuzzifier,
                 solve_strategy=(solve_optimization_tensorflow, {}),
                 random_state=None,
                 return_vars=False,
                 return_profile=False):
        self.c = c
        self.k = k
        self.sample_generator = sample_generator
        self.fuzzifier = fuzzifier
        self.solve_strategy = solve_strategy
        self.random_state = random_state
        self.return_vars = return_vars
        self.return_profile = return_profile

    def fit(self, X, y, **kwargs):

        check_X_y(X, y)
        self.random_state_ = check_random_state(self.random_state)

        if 'warm_start' in kwargs and kwargs['warm_start']:
            check_is_fitted(self, ['chis_', 'estimated_membership_'])
            self.solve_strategy[1]['init'] = self.chis_

        self.chis_ = solve_optimization(X, y,
                                        self.c, self.k,
                                        self.solve_strategy[0],
                                        **self.solve_strategy[1])

        if type(self.k) is kernel.PrecomputedKernel:
            self.gram_ = self.k.kernel_computations
        else:
            self.gram_ = np.array([[self.k.compute(x1, x2) for x1 in X]
                                    for x2 in X])
        self.fixed_term_ = np.array(self.chis_).dot(self.gram_.dot(self.chis_))

        def x_to_sq_dist(x_new):
            ret = self.k.compute(x_new, x_new) \
                  - 2 * np.array([self.k.compute(x_i, x_new)
                                  for x_i in X]).dot(self.chis_) \
                  + self.fixed_term_
            return ret
        self.x_to_sq_dist_ = x_to_sq_dist

        self.chi_SV_index_ = [i for i, (chi, mu) in enumerate(zip(self.chis_,
                                                                  y))
                              if -self.c * (1-mu) < chi < self.c * mu]
        
        print(X[self.chi_SV_index_])

        #self.chi_SV_index_ = [i for i in range(len(self.chis)_) \
        #        if -self.c*(1-self.mu[i]) < self.chis_[i] < self.c*self.mu[i]]

        chi_SV_square_distance = map(x_to_sq_dist, X[self.chi_SV_index_])
        chi_SV_square_distance = list(chi_SV_square_distance)
        #chi_SV_square_distance = [x_to_sq_dist(x[i]) for i in chi_SV_index]

        if len(chi_SV_square_distance) == 0:
            self.estimated_membership_ = None
            self.train_error_ = np.inf
            self.chis_ = None
            self.profile = None
            warn('No support vectors found')
            return self
            #raise ValueError('No support vectors found')

        self.SV_square_distance_ = np.mean(chi_SV_square_distance)
        num_samples = 500

        if self.sample_generator is None:
            self.sample_generator = lambda x: x

        #sample = map(self.sample_generator,
        #             self.random_state_.random_sample(num_samples))
        sample = self.sample_generator(num_samples)


        fuzzifier = self.fuzzifier(X, y)
        result = fuzzifier.get_membership(
                self.SV_square_distance_,
                sample,
                self.x_to_sq_dist_,
                return_profile=self.return_profile)


        if self.return_profile:
            self.estimated_membership_, self.profile_ = result
        else:
            self.estimated_membership_ = result[0]

        self.train_error_ = np.mean([(self.estimated_membership_(x) - mu)**2
                                    for x, mu in zip(X, y)])

        return self


    def predict(self, X):

        check_is_fitted(self, ['chis_', 'estimated_membership_'])
        X = check_array(X)
        return np.array([self.estimated_membership_(x) for x in X])

    def score(self, X, y):
        if self.estimated_membership_ is None:
            return -np.inf
        else:
            return -np.mean([(self.estimated_membership_(x) - mu)**2
                             for x, mu in zip(X, y)])

In [ ]:
xs  = np.array([[1], [2], [3], [4], [5],  [6], [7], [8], [9]])
mus = [1,     1,  .8,  .7,  .4,   .2,   0,   0,  0]

f = FuzzyInductor()

In [ ]:
f.fit(xs, mus)

100%|██████████| 100/100 [00:01<00:00, 50.39it/s]


[[1]
 [2]
 [3]
 [4]
 [5]
 [6]]


TypeError: get_membership() got multiple values for argument 'return_profile'